# Análise de Absentismo - Perfis Comportamentais

## Objetivo
Identificar e caracterizar **PERFIS** de colaboradores por comportamento de absentismo,
permitindo ações dirigidas, mensuráveis e eficazes.

## Metodologia
Em vez de listar estatísticas genéricas, agrupamos colaboradores ativos em **PERFIS**
com características comportamentais distintas - inspirado na análise TAAG que funcionou bem.

---

## 📋 Nota Importante

Este notebook foca em **perfis comportamentais**, não em estatísticas agregadas.

**Pré-requisito**: Execute primeiro o notebook `caracterização.ipynb` para preparar os dados.

Ou execute as células de preparação abaixo.

In [ ]:
# Imports e configurações
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')
plt.rcParams['figure.figsize'] = (14, 6)

print('✅ Setup concluído!')

## 1. Preparação dos Dados

**Opção A**: Carregar dados já preparados do notebook `caracterização.ipynb`

**Opção B**: Executar células de preparação abaixo (copiar do outro notebook)

Para esta análise precisamos do dataset agregado (`df_agregado`) e da tabela de métricas
por colaborador ativo (`metricas_colaborador`).

## 2. Criação de Perfis Comportamentais

Vamos segmentar colaboradores ativos em **5 PERFIS** baseados em comportamento:

- **PERFIL 1: O Exemplar** - Raramente falta, sempre pontual
- **PERFIL 2: O Atrasado** - Atrasos frequentes, mas raramente falta
- **PERFIL 3: O Ausente Ocasional** - Algumas faltas, geralmente justificadas
- **PERFIL 4: O Faltoso Estratégico** - Padrão de faltas às segundas/sextas
- **PERFIL 5: O Caso Crítico** - Faltas injustificadas frequentes ou padrões graves

### Critérios de Classificação
Usamos regras simples e interpretáveis (não clustering) para máxima clareza.

In [ ]:
# ASSUMINDO que 'metricas_colaborador' já existe (do notebook anterior)
# Se não existir, precisa executar células de preparação primeiro

def classificar_perfil(row):
    """
    Classifica colaborador em um dos 5 perfis baseado em regras.
    Ordem de prioridade: Crítico > Estratégico > Atrasado > Ausente > Exemplar
    """
    
    # PERFIL 5: Caso Crítico (prioridade máxima)
    if row['faltas_injustificadas'] >= 10:
        return 5
    
    # PERFIL 4: Faltoso Estratégico (≥30% faltas às segundas OU sextas, mínimo 5 problemas)
    if row['total_problemas'] >= 5 and (row['pct_faltas_segundas'] >= 30 or row['pct_faltas_sextas'] >= 30):
        return 4
    
    # PERFIL 2: Atrasado (≥10 atrasos, poucas faltas)
    if row['atrasos'] >= 10 and row['total_faltas'] <= 5:
        return 2
    
    # PERFIL 3: Ausente Ocasional (entre 3 e 15 faltas, maioria justificadas)
    if 3 <= row['total_faltas'] <= 15 and row['faltas_justificadas'] > row['faltas_injustificadas']:
        return 3
    
    # PERFIL 1: Exemplar (tudo o resto - baixo absentismo)
    return 1

# Aplicar classificação
print('Classificando colaboradores em perfis...')
metricas_colaborador['perfil'] = metricas_colaborador.apply(classificar_perfil, axis=1)

# Distribuição de perfis
print('\n' + '='*80)
print('📊 DISTRIBUIÇÃO DE PERFIS')
print('='*80)

dist_perfis = metricas_colaborador['perfil'].value_counts().sort_index()
total_ativos = len(metricas_colaborador)

nomes_perfis = {
    1: 'Exemplar',
    2: 'Atrasado',
    3: 'Ausente Ocasional',
    4: 'Faltoso Estratégico',
    5: 'Caso Crítico'
}

for perfil, count in dist_perfis.items():
    pct = count / total_ativos * 100
    print(f'  PERFIL {perfil} - {nomes_perfis[perfil]}: {count:,} colaboradores ({pct:.1f}%)')

print('\n✅ Classificação concluída!')

## 3. Caracterização Detalhada por Perfil

Para cada perfil, apresentamos:
- **Quantidade** de colaboradores
- **Características** comportamentais médias
- **Onde estão** concentrados (operações/segmentos)
- **Impacto** estimado
- **Recomendações** de ação

In [ ]:
# Função para caracterizar um perfil
def caracterizar_perfil(perfil_num, nome_perfil, dados):
    print('='*80)
    print(f'📊 PERFIL {perfil_num}: {nome_perfil.upper()}')
    print('='*80)
    
    # Filtrar colaboradores deste perfil
    perfil_data = dados[dados['perfil'] == perfil_num]
    n_colab = len(perfil_data)
    pct_total = n_colab / len(dados) * 100
    
    print(f'\n1️⃣ QUANTIDADE')
    print(f'   • {n_colab:,} colaboradores ({pct_total:.1f}% dos ativos)')
    
    print(f'\n2️⃣ CARACTERÍSTICAS COMPORTAMENTAIS (Médias)')
    print(f'   • Faltas injustificadas: {perfil_data["faltas_injustificadas"].mean():.1f}')
    print(f'   • Faltas justificadas: {perfil_data["faltas_justificadas"].mean():.1f}')
    print(f'   • Atrasos: {perfil_data["atrasos"].mean():.1f}')
    print(f'   • Taxa de absentismo: {perfil_data["taxa_absentismo"].mean():.1f}%')
    print(f'   • % faltas às segundas: {perfil_data["pct_faltas_segundas"].mean():.1f}%')
    print(f'   • % faltas às sextas: {perfil_data["pct_faltas_sextas"].mean():.1f}%')
    
    print(f'\n3️⃣ ONDE ESTÃO (Top 5 Operações)')
    top_ops = perfil_data['operacao'].value_counts().head(5)
    for op, count in top_ops.items():
        pct = count / n_colab * 100
        print(f'   • {op}: {count} ({pct:.1f}%)')
    
    print(f'\n4️⃣ IMPACTO ESTIMADO')
    total_prob = perfil_data['total_problemas'].sum()
    print(f'   • Total de dias perdidos (faltas + atrasos): {total_prob:,} dias')
    print(f'   • Equivalente a ~{total_prob/220:.0f} anos de trabalho (considerando 220 dias úteis/ano)')
    
    print('\n' + '='*80 + '\n')

# Caracterizar cada perfil
nomes_perfis = {
    1: 'O Exemplar',
    2: 'O Atrasado',
    3: 'O Ausente Ocasional',
    4: 'O Faltoso Estratégico',
    5: 'O Caso Crítico'
}

for perfil_num in sorted(nomes_perfis.keys()):
    if perfil_num in metricas_colaborador['perfil'].values:
        caracterizar_perfil(perfil_num, nomes_perfis[perfil_num], metricas_colaborador)

## 4. Visualizações - Perfis em Contexto

In [ ]:
# Visualização 1: Distribuição de perfis (Pizza + Barras)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Pizza
dist_perfis = metricas_colaborador['perfil'].value_counts().sort_index()
labels = [f'P{i}: {nomes_perfis[i]}' for i in dist_perfis.index]
colors = ['#2ecc71', '#f39c12', '#3498db', '#e67e22', '#e74c3c']
axes[0].pie(dist_perfis.values, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors[:len(dist_perfis)])
axes[0].set_title('Distribuição de Colaboradores por Perfil', fontsize=14, fontweight='bold')

# Barras horizontais
axes[1].barh(labels, dist_perfis.values, color=colors[:len(dist_perfis)])
axes[1].set_xlabel('Número de Colaboradores')
axes[1].set_title('Contagem por Perfil', fontsize=14, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Visualização 2: Comparação de métricas por perfil
metricas_comp = metricas_colaborador.groupby('perfil')[[
    'faltas_injustificadas', 'faltas_justificadas', 'atrasos', 'taxa_absentismo'
]].mean()

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

metricas_nomes = {
    'faltas_injustificadas': 'Faltas Injustificadas (média)',
    'faltas_justificadas': 'Faltas Justificadas (média)',
    'atrasos': 'Atrasos (média)',
    'taxa_absentismo': 'Taxa de Absentismo (%)'
}

for i, (col, titulo) in enumerate(metricas_nomes.items()):
    metricas_comp[col].plot(kind='bar', ax=axes[i], color=colors[:len(metricas_comp)])
    axes[i].set_title(titulo, fontsize=12, fontweight='bold')
    axes[i].set_xlabel('Perfil')
    axes[i].set_ylabel('Valor')
    axes[i].grid(axis='y', alpha=0.3)
    axes[i].set_xticklabels([f'P{p}' for p in metricas_comp.index], rotation=0)

plt.tight_layout()
plt.show()

## 5. Recomendações por Perfil

Ações específicas e mensuráveis para cada perfil.

In [ ]:
print('='*80)
print('💡 RECOMENDAÇÕES POR PERFIL')
print('='*80)

recomendacoes = {
    1: {
        'titulo': 'PERFIL 1: O Exemplar',
        'acao': 'Manter e Reconhecer',
        'passos': [
            'Não requer ação disciplinar',
            'Considerar para programas de reconhecimento',
            'Usar como referência de boas práticas',
            'Monitorizar para detectar mudanças de comportamento'
        ]
    },
    2: {
        'titulo': 'PERFIL 2: O Atrasado',
        'acao': 'Intervenção Preventiva',
        'passos': [
            'Implementar sistema de alertas automático após 3 atrasos consecutivos',
            'Reunião com supervisor para identificar causas (transporte, horário, etc.)',
            'Oferecer flexibilidade de horário quando possível',
            'Escalonar para ação disciplinar após 15 atrasos sem justificação'
        ]
    },
    3: {
        'titulo': 'PERFIL 3: O Ausente Ocasional',
        'acao': 'Monitorização e Suporte',
        'passos': [
            'Verificar se ausências são concentradas (pode indicar problema específico)',
            'Reunião de check-in trimestral com RH',
            'Oferecer suporte se padrão médico (muitas baixas)',
            'Estabelecer meta de redução de 20% no próximo semestre'
        ]
    },
    4: {
        'titulo': 'PERFIL 4: O Faltoso Estratégico',
        'acao': 'Intervenção Imediata',
        'passos': [
            'Reunião disciplinar OBRIGATÓRIA com gestor + RH',
            'Apresentar dados concretos do padrão identificado',
            'Carta de advertência formal',
            'Plano de melhoria com metas claras (max 2 faltas/mês)',
            'Reavaliação mensal obrigatória'
        ]
    },
    5: {
        'titulo': 'PERFIL 5: O Caso Crítico',
        'acao': 'Processo Disciplinar',
        'passos': [
            'Início IMEDIATO de processo disciplinar',
            'Reunião formal com presença de representante legal se aplicável',
            'Avaliar rescisão de contrato por justa causa',
            'Documentar todos os incidentes detalhadamente',
            'Considerar período de prova de 30 dias como última oportunidade'
        ]
    }
}

for perfil_num, info in recomendacoes.items():
    if perfil_num in metricas_colaborador['perfil'].values:
        n_colab = (metricas_colaborador['perfil'] == perfil_num).sum()
        print(f'\n{info["titulo"]} ({n_colab} colaboradores)')
        print(f'Ação: {info["acao"]}')
        print('Passos:')
        for i, passo in enumerate(info['passos'], 1):
            print(f'  {i}. {passo}')
        print()

print('='*80)

## 6. Sumário Executivo

Visão consolidada para apresentação à direção.

In [ ]:
print('='*80)
print('📊 SUMÁRIO EXECUTIVO - PERFIS DE ABSENTISMO')
print('='*80)

total_ativos = len(metricas_colaborador)
dist_perfis = metricas_colaborador['perfil'].value_counts().sort_index()

print(f'\n1️⃣ PANORAMA GERAL')
print(f'   • Total de colaboradores ativos analisados: {total_ativos:,}')
print(f'   • Identificados 5 perfis comportamentais distintos')
print(f'   • Período de análise: 18 meses (Jan 2024 - Jun 2025)')

print(f'\n2️⃣ DISTRIBUIÇÃO DOS PERFIS')
for perfil in sorted(dist_perfis.index):
    count = dist_perfis[perfil]
    pct = count / total_ativos * 100
    nome = nomes_perfis[perfil]
    print(f'   • Perfil {perfil} ({nome}): {count:,} ({pct:.1f}%)')

# Calcular colaboradores que requerem atenção (perfis 2, 3, 4, 5)
atencao = metricas_colaborador[metricas_colaborador['perfil'].isin([2, 3, 4, 5])]
pct_atencao = len(atencao) / total_ativos * 100

print(f'\n3️⃣ FOCO DE ATENÇÃO')
print(f'   • Colaboradores que requerem alguma forma de atenção: {len(atencao):,} ({pct_atencao:.1f}%)')
print(f'   • Destes, casos críticos (Perfil 4 e 5): {len(metricas_colaborador[metricas_colaborador["perfil"].isin([4, 5])]):,}')

# Impacto
dias_perdidos_atencao = atencao['total_problemas'].sum()
print(f'\n4️⃣ IMPACTO ESTIMADO (Perfis que Requerem Atenção)')
print(f'   • Dias perdidos (faltas + atrasos): {dias_perdidos_atencao:,} dias')
print(f'   • Equivalente a: {dias_perdidos_atencao/220:.0f} anos de trabalho')

print(f'\n5️⃣ PRÓXIMOS PASSOS RECOMENDADOS')
print('   1. Revisar lista de colaboradores Perfil 5 (Crítico) - Ação imediata')
print('   2. Iniciar reuniões disciplinares com Perfil 4 (Estratégico)')
print('   3. Implementar sistema de alertas para Perfil 2 (Atrasado)')
print('   4. Agendar check-ins com Perfil 3 (Ausente Ocasional)')
print('   5. Reconhecer publicamente Perfil 1 (Exemplar)')

print('\n✅ ANÁLISE CONCLUÍDA')
print('='*80)

---

## 📥 Exportação de Dados

Para facilitar o trabalho dos RH, podemos exportar listas de colaboradores por perfil.

In [ ]:
# Exportar colaboradores por perfil (opcional)
# Descomente as linhas abaixo para exportar

# for perfil_num in range(1, 6):
#     colaboradores_perfil = metricas_colaborador[metricas_colaborador['perfil'] == perfil_num]
#     filename = f'perfil_{perfil_num}_{nomes_perfis[perfil_num].replace(" ", "_")}.csv'
#     colunas_export = ['login', 'nome', 'operacao', 'segmento', 
#                       'faltas_injustificadas', 'faltas_justificadas', 'atrasos',
#                       'taxa_absentismo', 'pct_faltas_segundas', 'pct_faltas_sextas']
#     colaboradores_perfil[colunas_export].to_csv(filename, index=False, encoding='utf-8-sig')
#     print(f'✅ Exportado: {filename} ({len(colaboradores_perfil)} colaboradores)')

print('💡 Descomente o código acima para exportar listas por perfil')